# PROJECT 1 MODEL
# Brilliant

# STEP #1: IMPORTING DATA

In [1]:
#Importing Libraries 
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import time

In [ ]:
#importing data
dataset = pd.read_csv('data/new_appdata10.csv')